In [ ]:
import pickle
with open('E:\PaperCode\对标GraphBind\Datasets\PMG\PDB_DF\\6HMZ_X.csv.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    print(loaded_data)

In [ ]:
print(loaded_data['pdb_DF']['is_sidechain'])

In [ ]:
import numpy as np

# 假设有两个点的坐标
pointz = np.array([ 47.66925748 , 82.32201651 ,121.98438082])
point1 = np.array([ 47.72064049,  86.20137922, 122.56441245])-pointz  # 第一个点的坐标
point2 = np.array([ 51.50852779 , 87.40417973, 119.62607287])-pointz  # 第二个点的坐标
print(point1)
# 计算两个点的向量
vector1 = point1
vector2 = point2

# 计算点积
dot_product = np.dot(vector1, vector2)

# 计算向量的模长
magnitude1 = np.linalg.norm(vector1)
magnitude2 = np.linalg.norm(vector2)

# 计算夹角的余弦值
cosine_theta = dot_product / (magnitude1 * magnitude2)

# 计算夹角（弧度）
theta_rad = np.arccos(cosine_theta)

# 将弧度转换为度
theta_deg = np.degrees(theta_rad)

print("夹角 θ（度）:", theta_deg)


In [ ]:
np.sqrt(np.sum(np.square(point2)))

In [ ]:
import pandas as pd
path = 'E:/OwnCode/PionicNet/data/fasta/ionic/re/'
ionic_list = ['ZN', 'MG', 'CA', 'MN', 'FE', 'CU', 'FE2', 'NA', 'K', 'NI', 'G']
df = pd.DataFrame(columns=['ionic','number'])
for ionic in ionic_list:
    with open(path+ionic+'_label.fa','r') as file:
        df.loc[len(df)] = [ionic,int(len(file.readlines())/2)]
df

In [18]:
# 统计长度数据
import pandas as pd
path = 'E:/OwnCode/PionicNet/data/fasta/ionic/re/'
ionic_list = ['ZN', 'MG', 'CA', 'MN', 'FE', 'CU', 'FE2', 'NA', 'K', 'NI', 'G']
df = pd.DataFrame(columns=['ionic','number','0-50','50-1000','>1000'])
for ionic in ionic_list:
    with open(path+ionic+'_label.fa','r') as file:
        lines = file.readlines()
        number = int(len(lines)/2)
        lt50 = 0
        in51 = 0
        gt10 = 0
        for line in lines:
            if not line.startswith('>'):
                if len(line)-1<=50:
                    lt50 += 1
                elif len(line)-1>=1000:
                    gt10 +=1
                elif len(line)-1<1000 and len(line)-1>50:
                    in51 +=1
        df.loc[len(df)] = [ionic,number,lt50,in51,gt10]
df

,ionic,number,0-50,50-1000,>1000
0,ZN,44608,1638,40804,2166
1,MG,36017,305,34100,1612
2,CA,24358,141,23596,621
3,MN,8539,22,8363,154
4,FE,5353,3,5342,8
5,CU,3804,1,3787,16
6,FE2,2815,0,2813,2
7,NA,660,0,631,29
8,K,369,0,369,0
9,NI,328,0,328,0


In [9]:

import os

def get_all_subdirectories(directory):
    if  '.git' in directory:
        return []
    subdirectories = [f.path for f in os.scandir(directory) if f.is_dir()]
    for subdir in subdirectories:
        subdirectories.extend(get_all_subdirectories(subdir))
    return subdirectories

# 指定要获取子文件夹的目录路径
directory_path = 'J:/OwnCode/PionicNet'

# 获取所有子文件夹的列表
all_subdirectories = get_all_subdirectories(directory_path)

# 打印子文件夹列表
for subdir in all_subdirectories:
    print(subdir)



J:/OwnCode/PionicNet\.git
J:/OwnCode/PionicNet\.idea
J:/OwnCode/PionicNet\code
J:/OwnCode/PionicNet\.idea\inspectionProfiles
J:/OwnCode/PionicNet\code\data
J:/OwnCode/PionicNet\code\model
J:/OwnCode/PionicNet\code\temp
J:/OwnCode/PionicNet\code\model\__pycache__
J:/OwnCode/PionicNet\code\model\__pycache__
